In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
172,AFG,Asia,Afghanistan,2020-06-30,31238.0,271.0,733.0,12.0,802.449,6.962,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
286,ALB,Europe,Albania,2020-06-30,2466.0,64.0,58.0,3.0,856.905,22.239,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
464,DZA,Africa,Algeria,2020-06-30,13571.0,298.0,905.0,8.0,309.480,6.796,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-29,France,163151.0,0.0,29782.0,0.0,0.0,0.0
0,2020-06-29,China,84811.0,0.0,4651.0,0.0,0.0,0.0
0,2020-06-29,Italy,240625.0,0.0,34772.0,0.0,0.0,0.0
0,2020-06-29,Spain,249370.0,0.0,28362.0,0.0,0.0,0.0
0,2020-06-29,United States,2589665.0,0.0,126511.0,0.0,0.0,0.0
0,2020-06-29,World,10315543.0,0.0,507077.0,0.0,0.0,0.0
0,2020-06-29,United Kingdom,312439.0,0.0,43705.0,0.0,0.0,0.0
0,2020-06-29,Germany,194112.0,0.0,8974.0,0.0,0.0,0.0
0,2020-06-29,Iran,225675.0,0.0,10705.0,0.0,0.0,0.0
0,2020-06-29,Turkey,198510.0,0.0,5113.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-30,France,163151.0,164260.0,29782.0,29813.0,-1109.0,-31.0
0,2020-06-30,China,84811.0,84780.0,4651.0,4641.0,31.0,10.0
0,2020-06-30,Italy,240625.0,240436.0,34772.0,34744.0,189.0,28.0
0,2020-06-29,Spain,249370.0,248970.0,28362.0,28346.0,400.0,16.0
0,2020-06-30,United States,2589665.0,2590552.0,126511.0,126140.0,-887.0,371.0
0,2020-06-30,World,10315543.0,10273001.0,507077.0,505295.0,42542.0,1782.0
0,2020-06-30,United Kingdom,312439.0,311965.0,43705.0,43575.0,474.0,130.0
0,2020-06-30,Germany,194112.0,194259.0,8974.0,8973.0,-147.0,1.0
0,2020-06-30,Iran,225675.0,225205.0,10705.0,10670.0,470.0,35.0
0,2020-06-30,Turkey,198510.0,198613.0,5113.0,5115.0,-103.0,-2.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")